# word2vec

In [14]:
import json
from collections import Counter, OrderedDict
import numpy as np
import random
import math

## build dictionary

In [15]:
def LearnVocabFromTrainFile():
    f = open("poem.txt")
    vcount = Counter()
    for line in f.readlines():
        for w in line.decode("utf-8").strip().split():
            vcount.update(w)
    vcount_list = sorted(filter(lambda x: x[1] >= 5, vcount.items())
                         , reverse=True, key=lambda x: x[1])
    vocab_dict = OrderedDict(map(lambda x: (x[1][0], x[0]), enumerate(vcount_list)))
    vocab_freq_dict = OrderedDict(map(lambda x: (x[0], x[1]), vcount_list))
    return vocab_dict, vocab_freq_dict


In [16]:
vocab_dict, vocab_freq_dict =  LearnVocabFromTrainFile()

In [19]:
for w,wid in vocab_dict.items():
    print "%s : %s"%(w,wid)

不 : 0
人 : 1
山 : 2
無 : 3
風 : 4
一 : 5
日 : 6
雲 : 7
有 : 8
何 : 9
來 : 10
天 : 11
中 : 12
時 : 13
花 : 14
上 : 15
春 : 16
水 : 17
月 : 18
相 : 19
年 : 20
為 : 21
生 : 22
君 : 23
長 : 24
心 : 25
自 : 26
如 : 27
知 : 28
白 : 29
歸 : 30
此 : 31
秋 : 32
見 : 33
行 : 34
去 : 35
清 : 36
夜 : 37
江 : 38
在 : 39
空 : 40
下 : 41
得 : 42
高 : 43
未 : 44
明 : 45
多 : 46
金 : 47
門 : 48
青 : 49
是 : 50
客 : 51
處 : 52
寒 : 53
子 : 54
落 : 55
聲 : 56
玉 : 57
千 : 58
三 : 59
家 : 60
今 : 61
事 : 62
雨 : 63
道 : 64
遠 : 65
南 : 66
朝 : 67
前 : 68
萬 : 69
出 : 70
路 : 71
飛 : 72
入 : 73
我 : 74
東 : 75
城 : 76
草 : 77
深 : 78
與 : 79
盡 : 80
流 : 81
新 : 82
煙 : 83
開 : 84
樹 : 85
思 : 86
別 : 87
已 : 88
地 : 89
回 : 90
酒 : 91
將 : 92
欲 : 93
西 : 94
誰 : 95
馬 : 96
色 : 97
還 : 98
應 : 99
可 : 100
獨 : 101
成 : 102
聞 : 103
光 : 104
更 : 105
向 : 106
重 : 107
陽 : 108
同 : 109
身 : 110
雪 : 111
石 : 112
望 : 113
香 : 114
看 : 115
滿 : 116
愁 : 117
海 : 118
老 : 119
書 : 120
情 : 121
林 : 122
從 : 123
衣 : 124
古 : 125
歌 : 126
紅 : 127
頭 : 128
難 : 129
塵 : 130
裏 : 131
方 : 132
樓 : 133
龍 : 134
莫 : 135
舊 : 136
能 : 137
似 : 13

In [20]:
for w,wfreq in vocab_freq_dict.items():
    print "%s : %s"%(w,wfreq)

不 : 26426
人 : 20966
山 : 16056
無 : 15795
風 : 15618
一 : 15212
日 : 14952
雲 : 13382
有 : 12705
何 : 12385
來 : 12296
天 : 12173
中 : 11397
時 : 11342
花 : 11282
上 : 11131
春 : 11071
水 : 11046
月 : 10947
相 : 9660
年 : 9572
為 : 9545
生 : 9393
君 : 9305
長 : 9152
心 : 9052
自 : 9009
如 : 8738
知 : 8661
白 : 8515
歸 : 8338
此 : 8119
秋 : 8095
見 : 8077
行 : 7946
去 : 7767
清 : 7522
夜 : 7473
江 : 7432
在 : 7416
空 : 7360
下 : 7344
得 : 7311
高 : 7194
未 : 6973
明 : 6810
多 : 6594
金 : 6592
門 : 6541
青 : 6479
是 : 6455
客 : 6450
處 : 6439
寒 : 6303
子 : 6234
落 : 6202
聲 : 6170
玉 : 6134
千 : 6076
三 : 6040
家 : 6003
今 : 5983
事 : 5966
雨 : 5945
道 : 5897
遠 : 5888
南 : 5772
朝 : 5762
前 : 5748
萬 : 5742
出 : 5648
路 : 5568
飛 : 5536
入 : 5519
我 : 5501
東 : 5497
城 : 5445
草 : 5415
深 : 5374
與 : 5319
盡 : 5301
流 : 5296
新 : 5295
煙 : 5282
開 : 5267
樹 : 5257
思 : 5249
別 : 5086
已 : 5079
地 : 5044
回 : 5033
酒 : 4991
將 : 4982
欲 : 4979
西 : 4977
誰 : 4971
馬 : 4971
色 : 4969
還 : 4941
應 : 4848
可 : 4803
獨 : 4759
成 : 4747
聞 : 4716
光 : 4665
更 : 4600
向 : 4598
重 : 4592
陽 : 4499


## build unigram table (for negative sampling)

In [21]:
def InitUnigramTable(vocab_freq_dict):
    table_freq_list = map(lambda x: (x[0], int(x[1][1] ** 0.75)), enumerate(vocab_freq_dict.items()))
    table_size = sum([x[1] for x in table_freq_list])
    table = np.zeros(table_size).astype(int)
    offset = 0
    for item in table_freq_list:
        table[offset:offset + item[1]] = item[0]
        offset += item[1]

    return table


In [22]:
table = InitUnigramTable(vocab_freq_dict)

In [25]:
np.set_printoptions(threshold=20)
print table

[   0    0    0 ..., 5499 5499 5499]


In [39]:
def train(vocab_dict, vocab_freq_dict, table):
    total_words = sum([x[1] for x in vocab_freq_dict.items()])
    vocab_size = len(vocab_dict)
    layer1_size = 30
    window = 2
    alpha_init = 0.025
    sample = 0.001
    negative = 10
    train_words = 0
    ite = 2
    syn0 = (0.5 - np.random.rand(vocab_size, layer1_size)) / layer1_size
    syn1 = np.zeros((layer1_size, vocab_size))
    p_count = 0
    avg_err = 0.
    err_count = 0
    for local_iter in range(ite):
        print "local_iter", local_iter
        f = open("poem.txt")
        for line in f.readlines():
            sen = []
            for word_raw in line.decode("utf-8").strip().split():
                last_word = vocab_dict.get(word_raw, -1)
                if last_word == -1:
                    continue
                cn = vocab_freq_dict.get(word_raw)
                ran = (math.sqrt(cn / float(sample * total_words + 1))) * (sample * total_words) / cn
                if ran < random.random():
                    continue
                train_words += 1
                sen.append(last_word)
            alpha = alpha_init * (1 - train_words / float(ite * total_words + 1))
            if alpha < alpha_init * 0.0001:
                alpha = alpha_init * 0.0001
            for a, word in enumerate(sen):
                b = random.randint(1, window)
                for c in range(a - b, a + b + 1):
                    if c < 0 or c == a or c >= len(sen):
                        continue
                    h_err = np.zeros((layer1_size))
                    last_word = sen[c]
                    for negcount in range(negative):
                        if negcount == 0:
                            target_word = word
                            label = 1
                        else:
                            while True:
                                target_word = table[random.randint(0, len(table) - 1)]
                                if target_word not in sen:
                                    break
                            label = 0
                        o_pred = 1 / (1 + np.exp(- np.dot(syn0[last_word, :], syn1[:, target_word])))
                        o_err = o_pred - label
                        h_err += o_err * syn1[:, target_word]
                        syn1[:, target_word] -= alpha * o_err * syn0[last_word]
                        avg_err += abs(o_err)
                        err_count += 1
                    syn0[last_word, :] -= alpha * h_err
                    p_count += 1
                    if p_count % 10000 == 0:
                        print "Iter: %s, Alpha %s, Train Words %s, Average Error: %s" \
                              % (local_iter, alpha, 100 * train_words, avg_err / float(err_count))
                        avg_err = 0.
                        err_count == 0.
        model_name = "w2v_model_%s.json" % (local_iter)
        print "save model: %s" % (model_name)
        fm = open(model_name, "w")
        fm.write(json.dumps(syn0.tolist(), indent=4))
        fm.close()

In [ ]:
train(vocab_dict, vocab_freq_dict, table)

## Show Result

In [40]:
f2 = open("w2v_model_1.json", "r")  # "w2v_result_1.json"
w2v_model = np.array(json.loads("".join(f2.readlines())))
f2.close()

vocab_dict_reversed = OrderedDict([(x[1], x[0]) for x in vocab_dict.items()])

def get_top(word):
    wid = vocab_dict.get(word)
    dot_result = np.dot(w2v_model, np.expand_dims(w2v_model[wid], axis=1))
    norm = np.sqrt(np.sum(np.power(w2v_model.T, 2), axis=0))
    cosine_result = np.divide(dot_result[:, 0], norm*norm[wid])
    final_result = sorted(filter(lambda x:x[0] != wid, [(x[0], x[1]) for x in enumerate(cosine_result)]),
                          key=lambda x: x[1], reverse=True)
    print word
    for x in final_result[:5]:
        print vocab_dict_reversed.get(x[0]), x[1]


In [41]:
get_top(u"山")
get_top(u"峰")
get_top(u"河")
get_top(u"日")
get_top(u"母")


山
嶺 0.854901128361
嵩 0.846620438864
峰 0.842831270385
岡 0.838129842909
嶂 0.834701215189
峰
山 0.842831270385
嶽 0.83917452917
嶺 0.8219837161
頂 0.821088331571
嶂 0.809565794884
河
湟 0.787726187693
涇 0.770652269018
淮 0.751135710239
川 0.742243126005
汾 0.740643816278
日
旦 0.869047480855
又 0.842383624714
曛 0.830549707539
夕 0.826327222048
暉 0.82616774597
母
阿 0.841291555189
粲 0.758949798908
父 0.757878716348
子 0.748425991864
薛 0.74610941665


In [42]:
def get_calculated_top(w1, w2, w3):
    wid1, wid2, wid3 = vocab_dict.get(w1), vocab_dict.get(w2), vocab_dict.get(w3)
    v1, v2, v3 = w2v_model[wid1], w2v_model[wid2], w2v_model[wid3]
    combined_vec = v1 + (v2 - v3)
    dot_result = np.dot(w2v_model, np.expand_dims(combined_vec, axis=1))
    norm = np.sqrt(np.sum(np.power(w2v_model.T, 2), axis=0))
    cvec_norm = np.sqrt(np.sum(np.power(combined_vec, 2)))
    cosine_result = np.divide(dot_result[:, 0], norm * cvec_norm)

    final_result = sorted(filter(lambda x: x[0] not in [wid1, wid2, wid3],
                                 [(x[0], x[1]) for x in enumerate(cosine_result)]),
                          key=lambda x: x[1], reverse=True)
    print "%s + %s - %s" % (w1, w2, w3)
    for x in final_result[:5]:
        print vocab_dict_reversed.get(x[0]), x[1]



In [43]:
get_calculated_top(u"女", u"父", u"男")

女 + 父 - 男
母 0.731002049447
娥 0.707469857054
客 0.69027387716
娃 0.687831493041
侶 0.681667240226
